In [1]:
pip install youtube-transcript-api whisper-openai transformers datasets accelerate langchain chromadb gradio

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 19.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Install required packages
!pip install -qU yt-dlp openai chromadb tiktoken pandas

In [13]:
# Import libraries
import yt_dlp
import os
from getpass import getpass
import pandas as pd
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import requests

In [14]:
# Step 1: Define a function to download transcripts
def download_transcripts(video_ids):
    transcripts = []
    ydl_opts = {
        'quiet': True,
        'writeautomaticsub': True,
        'subtitleslangs': ['en'],
        'skip_download': True,
    }
    for video_id in video_ids:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            try:
                info = ydl.extract_info(f"https://www.youtube.com/watch?v={video_id}", download=False)
                subtitles = info.get('requested_subtitles', {})
                if 'en' in subtitles:
                    transcripts.append({
                        'id': video_id,
                        'title': info.get('title', 'Unknown Title'),
                        'text': subtitles['en']['url']  # This gives a URL to download the transcript
                    })
            except Exception as e:
                print(f"Error downloading transcript for {video_id}: {e}")
    return transcripts

# Video IDs
video_ids = ['2u4ItZerRac', 'I2zF1I60hPg', '8xqSF-uHCUs', 'LtmS-c1pChY', 'sJNxT-I7L6s']

# Download transcripts
transcripts = download_transcripts(video_ids)

In [15]:
# Step 2: Download transcript text
for transcript in transcripts:
    try:
        response = requests.get(transcript['text'])  # Fetch the transcript text
        transcript['text'] = response.text
    except Exception as e:
        print(f"Error fetching transcript text for {transcript['id']}: {e}")

# Convert transcripts into a DataFrame
data = pd.DataFrame(transcripts)

In [16]:
# Step 3: Initialize OpenAI Embeddings
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or getpass("Enter your OpenAI API key: ")
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

C:\Users\mehak\AppData\Local\Temp\ipykernel_2464\2086685799.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embed = OpenAIEmbeddings(


In [17]:
# Step 4: Generate embeddings
data['embeddings'] = data['text'].apply(lambda x: embed.embed_documents([x])[0])


In [22]:
# Step 5: Set up ChromaDB
from chromadb import Client
from chromadb.config import Settings

# Initialize ChromaDB client
chroma_client = Client(
    Settings(
        persist_directory="./chromadb",  # Directory for database persistence
        anonymized_telemetry=False
    )
)

# Create or get a collection
collection = chroma_client.get_or_create_collection(name="youtube_transcripts")

# Step 6: Add embeddings to the collection
for _, row in data.iterrows():
    collection.add(
        ids=[row['id']],
        documents=[row['text']],
        metadatas=[{'title': row['title']}],
        embeddings=[row['embeddings']]
    )

# Persist the database
print("Data added successfully!")

Data added successfully!


In [24]:
# Step 6: Add embeddings to ChromaDB collection
for _, row in data.iterrows():
    collection.add(
        ids=[row['id']],  # The ID of the document
        documents=[row['text']],  # The document content
        metadatas=[{'title': row['title']}],  # Metadata (e.g., title)
        embeddings=[row['embeddings']]  # Embeddings generated earlier
    )

# Print success message
print("Embeddings successfully added to the collection!")

Insert of existing embedding ID: 2u4ItZerRac
Add of existing embedding ID: 2u4ItZerRac
Insert of existing embedding ID: I2zF1I60hPg
Add of existing embedding ID: I2zF1I60hPg
Insert of existing embedding ID: 8xqSF-uHCUs
Add of existing embedding ID: 8xqSF-uHCUs
Insert of existing embedding ID: LtmS-c1pChY
Add of existing embedding ID: LtmS-c1pChY
Insert of existing embedding ID: sJNxT-I7L6s
Add of existing embedding ID: sJNxT-I7L6s


Embeddings successfully added to the collection!


In [27]:
# Step 7: Query ChromaDB
query = "What are the benefits of immigration to Germany?"
results = collection.query(
    query_texts=[query],  # The query text
    n_results=3           # Number of top results to retrieve
)

# Print the results
print("Query Results:")
for res in results['documents']:
    print(res)


query = "What is the main topic of the video?"
results = collection.query(
    query_texts=[query],
    n_results=3
)
# Print the results
print("Query Results:")
for res in results['documents']:
    print(res)

InvalidDimensionException: Embedding dimension 384 does not match collection dimensionality 1536

In [ ]:

# Display results
for res in results['documents']:
    print(res)